In [1]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/language-detection/language-detection-data-v5.json.zip
# !unzip language-detection-data-v5.json.zip

In [2]:
import json

In [3]:
with open('language-detection-data-v5.json') as fopen:
    lang = json.load(fopen)

In [4]:
others = [text for no, text in enumerate(lang['text']) if lang['label'][no] not in ['zlm', 'ind', 'eng']]
len(others)

331597

In [5]:
eng = [text for no, text in enumerate(lang['text']) if lang['label'][no] == 'eng']
len(eng)

50000

In [6]:
zlm = [text for no, text in enumerate(lang['text']) if lang['label'][no] == 'zlm']
len(zlm)

53692

In [7]:
zlm_words = set(' '.join(zlm).split())

In [8]:
ind = [text for no, text in enumerate(lang['text']) if lang['label'][no] == 'ind']
len(ind)

57327

In [9]:
ind_words = set(' '.join(ind).split())

In [10]:
len(zlm_words), len(ind_words)

(17286, 7614)

In [11]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/200k-english-malay/200k-english-malay.json

In [12]:
with open('200k-english-malay.json') as fopen:
    english_malay = json.load(fopen)
    
malays = set([i[1] for i in english_malay])
len(malays)

150125

In [13]:
# !wget https://dl.fbaipublicfiles.com/arrival/dictionaries/en-ms.txt

In [14]:
with open('en-ms.txt') as fopen:
    en_ms = list(filter(None, fopen.read().split('\n')))
    
en_ms = [i.split('\t') for i in en_ms]
en_ms = set([i[1] for i in en_ms if i[1] != i[0]])

In [15]:
malays = malays | en_ms

In [16]:
with open('id-wiki.txt') as fopen:
    id_wiki = fopen.read().split('\n')
    
len(id_wiki)

393429

In [17]:
import re
from unidecode import unidecode
import cleaning
from tqdm import tqdm

def preprocessing(string):
    string = re.sub(
        'http\S+|www.\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )
    
    chars = ',.()!:\'"/;=-'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(
        u'[0-9!@#$%^&*()_\-+{}|\~`\'";:?/.>,<]',
        ' ',
        string,
        flags = re.UNICODE,
    )
    string = re.sub(r'[ ]+', ' ', string).strip()
    
    return string.lower()

def loop(strings):
    for i in tqdm(range(len(strings))):
        strings[i] = preprocessing(strings[i])
    return strings

In [18]:
id_wiki = cleaning.multiprocessing(id_wiki, loop)

100%|██████████| 24589/24589 [00:17<00:00, 1373.61it/s]


In [19]:
id_wiki_words = set(' '.join(id_wiki).split())
len(id_wiki_words)

1624903

In [20]:
with open('indon-news.json') as fopen:
    news = json.load(fopen)
    
indon_news = []

for n in news:
    indon_news.extend(n['text'].split('.'))

indon_news = list(filter(None, indon_news))

len(indon_news)

16353

In [21]:
indon_news = cleaning.multiprocessing(indon_news, loop)

indon_news_words = set(' '.join(indon_news).split())

100%|██████████| 1022/1022 [00:00<00:00, 28869.54it/s]

100%|██████████| 1022/1022 [00:00<00:00, 30735.87it/s]



100%|██████████| 1022/1022 [00:00<00:00, 30567.55it/s]


100%|██████████| 1022/1022 [00:00<00:00, 21817.31it/s]



In [22]:
ind_words = ind_words | id_wiki_words | indon_news_words

In [23]:
# !wget https://malaya-dataset.s3-ap-southeast-1.amazonaws.com/wikidump1-raw.json

In [24]:
with open('wikidump1-raw.json') as fopen:
    ms_wiki = json.load(fopen)
    
len(ms_wiki)

1748387

In [25]:
ms_wiki_words = set(' '.join(ms_wiki).split())
len(ms_wiki_words)

1893949

In [26]:
malays = malays | zlm_words | ms_wiki_words

In [27]:
with open('malays_word.json', 'w') as fopen:
    json.dump(list(malays), fopen)

In [28]:
intersected = malays.intersection(ind_words)

In [29]:
ms_wiki = cleaning.multiprocessing(ms_wiki, loop)

100%|██████████| 109274/109274 [00:03<00:00, 34344.70it/s]


In [30]:
ind_minus = ind_words - intersected
len(ind_minus)

1489582

In [31]:
import malaya

english = malaya.texts._english_words._english_words

In [32]:
for i in tqdm(range(len(ms_wiki))):
    ms_wiki[i] = ' '.join([w for w in ms_wiki[i].split() if w not in ind_minus and w not in english])

100%|██████████| 1748387/1748387 [00:10<00:00, 166086.88it/s]


In [33]:
for i in tqdm(range(len(zlm))):
    zlm[i] = ' '.join([w for w in zlm[i].split() if w not in ind_minus and w not in english])

100%|██████████| 53692/53692 [00:01<00:00, 31482.41it/s]


In [35]:
with open('ind_words.json', 'w') as fopen:
    json.dump(list(ind_minus), fopen)

In [36]:
with open('formal-language.json', 'w') as fopen:
    json.dump({'other': others,
              'malay': zlm + ms_wiki,
              'ind': ind + id_wiki,
              'eng': eng}, fopen)